# 04장 직류전원

* 이름: 홍길동
* 학번: 2025123456
* 조: 1

## 공통 코드

In [ ]:
import sys
from pathlib import Path

# common_dir = Path.cwd().parent / "ece2knu_common"
common_dir = Path.cwd()
sys.path.insert(0, str(common_dir))

from common import *

*******************************************************************************

## 4.3 실험 이론

### 4.3.5 평활회로

다음 회로도를 KiCad 로 그린다.

* 사용 소자
    + D: D1N4004
    + C: 10 uF (C_Polarized 사용)
    + RL: 5.1 K, 2k, 500 Ohm 

* 소자 reference (이름): KiCad 에서는 모든 소자 Reference 에 번호가 붙어야 하기 때문에, 각 소자의 Reference 는 다음과 같이 입력한다.
    + D: D1
    + C: C1
    + RL: RL1

* RL Parameter Sweep 설정
    + 소자 값을 다음과 같이 {<변수명>} 으로 입력 
        - 예: {Rvar}

    + 아래 control code 를 text 로 회로도에 추가

.param Rvar=5.1k

.control
    echo "x1 time x2 v(/vin) x3 v(/vout)" > ../../data/ch04_03_05_result_sim.txt

    foreach R 5.1k 2k 500
        alterparam Rvar = $R
        reset
        save time v(/vin) v(/vout)

        tran 64u 64m

        echo "Step Information:" >> ../../data/ch04_03_05_result_sim.txt
        set appendwrite
        wrdata ../../data/ch04_03_05_result_sim.txt time v(/vin) v(/vout)
    end
.endc

* 참고자료
    - https://ngspice.sourceforge.io/docs.html

In [ ]:
display_image("./images/ch04_03_03_05_circuit.png", width=700)

그린 회로도

In [ ]:
display_image("./data/ch04_03_03_05_circuit_cad.png", width=700)

####  회로 시뮬레이션하기

다음 설정으로 시뮬레이션한다.

In [ ]:
display_image("./images/ch04_03_03_05_sim_setting.png", width=700)

#### 시뮬레이션 결과 관찰한기

In [ ]:
display_image("./data/ch04_03_05_result_plot.png", width=700)

시뮬레이션한 데이터는 회로도에 입력한 control code 에 의해 자동으로 저장되므로, 직접 저장할 필요가 없다.

##### 결과 데이터 분석하기

그래프를 그린다.

In [ ]:
R_L_l = np.array([5100, 2000, 500])

# 실험 결과 파일을 읽는다.
data = get_simulation_result('data/ch04_03_05_result_sim.txt')

times_l = []
vins_l = []
vouts_l = []

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
for idx in range(len(R_L_l)):
    times_l.append(data['(TIME)@%d' % (idx+1)])
    vins_l.append(data['(V(/VIN))@%d' % (idx+1)] / 1000.)
    vouts_l.append(data['(V(/VOUT))@%d' % (idx+1)] / 1000.)

for idx in range(len(R_L_l)):
    R_L = R_L_l[idx]
    times = times_l[idx]
    vins = vins_l[idx]
    vouts = vouts_l[idx]
    label = 'V(OUT)(R_L=%d)' % R_L

    # 필터를 적용해 잡음을 줄인다.
    # vins = savgol_filter(vins, 100, 2)
    # vouts = savgol_filter(vouts, 100, 2)

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('time')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    # plt.axvline(0, color='black')
    if idx == 0:
        draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts, label, idx+1)

plt.show()